In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [3]:
df_pre = pd.read_csv('data/wine.csv')
df = df_pre.sample(frac=1)

# 1쓰면 전체 다 갖고 오는거 = 하나 마나인데, 일단 함 -> 셈플을 렌덤으로 가져옴
# 원래 데이터가 섞여서 들어오는 건 있음

# 전체 데이터중 70%만 쓰겠다 -> 0.7을 주면 -> 무작위로 섞어서 70%만 씀

In [4]:
print(df.head())

      7.4   0.7     0   1.9  0.076    11     34   0.9978  3.51  0.56   9.4  5  \
5315  6.3  0.18  0.24   3.4  0.053  20.0  119.0  0.99373  3.11  0.52   9.2  6   
2423  7.0  0.21  0.37   7.2  0.042  36.0  167.0  0.99580  3.26  0.56   9.8  6   
3517  7.3  0.22  0.50  13.7  0.049  56.0  189.0  0.99940  3.24  0.66   9.0  6   
1622  6.6  0.27  0.41   1.3  0.052  16.0  142.0  0.99510  3.42  0.47  10.0  6   
4060  8.3  0.25  0.33   2.5  0.053  12.0   72.0  0.99404  2.89  0.48   9.5  5   

      1  
5315  0  
2423  0  
3517  0  
1622  0  
4060  0  


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6496 entries, 5315 to 2732
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   7.4     6496 non-null   float64
 1   0.7     6496 non-null   float64
 2   0       6496 non-null   float64
 3   1.9     6496 non-null   float64
 4   0.076   6496 non-null   float64
 5   11      6496 non-null   float64
 6   34      6496 non-null   float64
 7   0.9978  6496 non-null   float64
 8   3.51    6496 non-null   float64
 9   0.56    6496 non-null   float64
 10  9.4     6496 non-null   float64
 11  5       6496 non-null   int64  
 12  1       6496 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 710.5 KB
None


In [6]:
dataset = df.values
print(dataset.dtype)

# 전체가 다 숫자(float)이니까 .astype 안해도 될듯

float64


In [7]:
X = dataset[:, :12]
Y = dataset[:, 12]

In [8]:
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
## 1이냐, 0이냐 맞추는 문제 = 2진 분류 = binary~~ -> sigmoid로 0,1이냐 뽑아주는것

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                390       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                372       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 875
Trainable params: 875
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [11]:
# https://thebook.io/080228/part04/ch14/02/

# 이제 모델을 저장하기 위해 케라스의 콜백 함수 중 ModelCheckpoint() 함수를 불러옵니다.
from keras.callbacks import ModelCheckpoint


# 해당 폴더의 존재 유무 확인
MODEL_DIR = './model/'   # # 모델을 저장하는 폴더
if not os.path.exists(MODEL_DIR):  # 만일 위의 폴더가 존재하지 않으면
    os.mkdir(MODEL_DIR)   # 이 이름의 폴더를 만들어 줌

# 미리 만듬
# epoch을 2자리 정수로 만들고, val값   , hdf5로 미리 만듬
modelpath = './model/{epoch:02d}-{val_loss:.4f}.hdf5'

# 체크포인트 생성
# 무엇을 체크할 것이냐 monitor = val_loss
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss',
                              verbose=1, save_best_only=True)
# save_best_only=True -> model이 향상될때만 저장

#Epoch 0~199까지 총 200개의 모델이 model 폴더에 저장되었습니다. 
#저장된 파일의 이름이 곧 에포크 수와 이때의 테스트셋 오차 값입니다. 
#이때 ModelCheckpoint() 함수에 모델이 앞서 저장한 모델보다 
#나아졌을 때만 저장하게끔 하려면 save_best_only 값을 True로 지정합니다.



# verbose=1 -> 중간과정 출력하라는 것

NameError: name 'os' is not defined

In [ ]:
# 위 처럼 놓고 fit을 수정
# model.fit(X, Y, epochs=200, batch_size=200)
model.fit(X, Y, validation_split=0.2, epochs=200, batch_size=200, 
         verbose=0, callbacks=[checkpointer]) #   varbose=0  -> fit 했을때 아래 그래프 출력 안나오게 함
#이제 모델을 학습할 때마다 
#위에서 정한 checkpointer의 값을 받아 지정된 곳에 모델을 저장합니다.

#print("Accuracy: %.4f" % (model.evaluate(X, Y[1])))